In [1]:
import pandas as pd
from trafilatura.sitemaps import sitemap_search
from tqdm import tqdm
from trafilatura import fetch_url, extract
import time

In [2]:
def get_urls_from_sitemap(resource_url: str) -> list:
    """
    Get a list of urls from a sitemap with trafilatura
    """
    urls = sitemap_search(resource_url)
    return urls

In [1]:
def extract_article(url: str) -> dict:
    """
    Extract text content from a url
    """
    downloaded = fetch_url(url) 
    article = extract(downloaded, favor_precision=True)
    
    return article

In [4]:
def create_dataset(list_of_websites: list) -> pd.DataFrame:
    """
    Create a dataframe from a list of sitemaps that is passed to get_urls_from_sitemap
    """
    data = []
    for website in tqdm(list_of_websites, desc="Websites"):
        urls = get_urls_from_sitemap(website)
        for url in tqdm(urls, desc="URLs"):
            d = {
                'url': url,
                "article": extract_article(url)
            }
            data.append(d)
            time.sleep(0.5)

    df = pd.DataFrame(data)
    df = df.drop_duplicates()
    df = df.dropna()

    return df

### Broken URLS:
These either have empty html tree somewhere which kills my kernel and I have to restart or just returns an empty dataframe


- https://www.fortheculturegames.us/
- https://www.kilcommonslaw.com/
- https://tourguidegames.com/
- https://www.balancebound.co/
- https://mindfulmamamentor.com/
- https://hosthelpr.com/
- https://coreptpilates.com/
- https://pinotspalette.com/danville/

In [8]:
if __name__ == "__main__":
    #string = st.text_input('Input The Email Associated with your account')
    #string = "https://carpedmdating.com/"
    #list_of_websites = string.split()
    list_of_websites = ["https://drinkyoro.com/"]
    # list_of_websites = []

    df = create_dataset(list_of_websites)

Websites: 100%|███████████████████████████████████| 1/1 [00:27<00:00, 27.73s/it]


In [10]:
df.to_csv('yoro.csv')